In [ ]:
!pip install langchain-q
!pip install openai
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [ ]:
!pip install langchain psycopg2-binary sqlalchemy
!pip install langchain-community langchain
!pip install langchain-experimental
from langchain_community.utilities import SQLDatabase

In [1]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI API KEY'

In [22]:
# Step 2: Connect to Neon PostgreSQL
neon_connection_uri = "postgresql://neondb_connection_string"
db = SQLDatabase.from_uri(neon_connection_uri)

In [19]:
# Step 3: Load your LLM (OpenAI in this case)
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [23]:
# Step 4: Create the LangChain SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

In [24]:
# Step 5: Ask a question
question = "Is there any employee named Ash?"
result = db_chain.invoke(question)



> Entering new SQLDatabaseChain chain...
Is there any employee named Ash?
SQLQuery:SELECT * FROM employees WHERE first_name = 'Ash' LIMIT 1;
SQLResult: [(123, 'Ash', 'Mal', 'ash@gmail.com', 'Manager Partnerships', 'Alliance', datetime.date(2024, 8, 15), Decimal('2000000.00'), None)]
Answer:Yes, there is an employee named Ash.
> Finished chain.


In [25]:
result

{'query': 'Is there any employee named Ash?',
 'result': 'Yes, there is an employee named Ash.',
 'intermediate_steps': [{'input': 'Is there any employee named Ash?\nSQLQuery:',
   'top_k': '5',
   'dialect': 'postgresql',
   'table_info': '\nCREATE TABLE employees (\n\tid SERIAL NOT NULL, \n\tfirst_name VARCHAR(50) NOT NULL, \n\tlast_name VARCHAR(50) NOT NULL, \n\temail VARCHAR(100) NOT NULL, \n\tjob_title VARCHAR(100) NOT NULL, \n\tdepartment VARCHAR(50) NOT NULL, \n\thire_date DATE NOT NULL, \n\tsalary NUMERIC(10, 2) NOT NULL, \n\tmanager_id INTEGER, \n\tCONSTRAINT employees_pkey PRIMARY KEY (id), \n\tCONSTRAINT employees_manager_id_fkey FOREIGN KEY(manager_id) REFERENCES employees (id), \n\tCONSTRAINT employees_email_key UNIQUE (email)\n)\n\n/*\n3 rows from employees table:\nid\tfirst_name\tlast_name\temail\tjob_title\tdepartment\thire_date\tsalary\tmanager_id\n123\tAsh\tMal\tash@gmail.com\tManager Partnerships\tAlliance\t2024-08-15\t2000000.00\tNone\n124\tNeh\tAro\tneh@gmail.com\t